In [1]:
local_dir = '/home/neilor/scielo_db/dados'
xml_dir = '/home/neilor/scielo_db/dados/artigos_xml'
connect_cmd = "dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'"
%load_ext sql
%sql postgresql://neilor:n1f2c3n1@/scielo_db

'Connected: neilor@scielo_db'

In [2]:
import wgetter
import time
import os
import pandas as pd
import psycopg2 as pg
from pathlib import Path
from lxml import etree
from xml.etree.ElementTree import ParseError

In [3]:
query_create_table_artigos_dados_xml = """
drop table if exists alacip.artigos_dados_xml;
create table alacip.artigos_dados_xml
(
    scielo_id varchar,
    titulo text,
    resumo text,
    palavra_chave1 varchar,
    palavra_chave2 varchar,
    palavra_chave3 varchar,
    palavra_chave4 varchar,
    palavra_chave5 varchar,
    empirico varchar,
    tem_tabela varchar,
    tem_quadro varchar,
    tem_grafico varchar,
    tem_diagrama varchar,
    tem_figura varchar,
    abstract text,
    key_word1 varchar,
    key_word2 varchar,
    key_word3 varchar,
    key_word4 varchar,
    key_word5 varchar,
    texto text
);

CREATE INDEX texto_idx ON alacip.artigos_dados_xml USING GIN (to_tsvector('portuguese', texto));
"""

%sql {query_create_table_artigos_dados_xml}


Done.
Done.
Done.


[]

In [4]:
url_xml = 'file:///home/neilor/scielo_db/dados/artigos_xml/S0011-52581998000300001.xml'
#url_xml = 'file:///home/neilor/scielo_db/dados/artigos_xml/S0104-62762016000100028.xml'
tree = etree.parse(url_xml)

def get_title(tree):
    r=tree.xpath('/article/front/article-meta/title-group/article-title')
    if r == []:
        return ''
    else:
        return r[0].text

def get_resumo(tree):
    r=tree.xpath('/article/front/article-meta/abstract[@xml:lang="pt"]')
    if r == []:
        return ''
    else:
        return(r[0][0].text)

def get_abstract(tree):
    r=tree.xpath('/article/front/article-meta/abstract[@xml:lang="en"]')
    if r == []:
        return ''
    else:
        return(r[0][0].text)

def get_palavras_chave(tree):
    r=tree.xpath('/article/front/article-meta/kwd-group/kwd[@lng="pt"]') 
    ke = []
    if r == []:
        return []
    else:
        for k in r:
            ke.append(k.text)
        return ke
    
def get_key_words(tree):
    r=tree.xpath('/article/front/article-meta/kwd-group/kwd[@lng="en"]') 
    ke = []
    if r == []:
        return []
    else:
        for k in r:
            ke.append(k.text)
        return ke

def get_autores(tree):
    n=tree.xpath('/article/front/article-meta/contrib-group/contrib[@contrib-type="author"]/name')
    nomes = []
    for i in n:
        nome = [i[0].text]+[i[1].text]
        nomes.append(nome)
    return nomes




def get_referencias_journal(tree):     
    print('========== ref ==================')
    r=tree.xpath('/article/back/ref-list/ref/nlm-citation[@citation-type="journal"]/*')
    nomes = []
    for i in r:
        print(i.text)
        #nome = [i[0].text]+[i[1].text]
        #nomes.append(nome)
    return nomes




def get_referencias_nomes(tree):     
    print('========== ref ==================')
    r=tree.xpath('/article/back/ref-list/ref/nlm-citation/person-group/name')
    nomes = []
    for i in r:
        nome = [i[0].text]+[i[1].text]
        nomes.append(nome)
    return nomes

def get_referencias_source(tree):     
    print('========== ref ==================')
    r=tree.xpath('/article/back/ref-list/ref/nlm-citation/source')
    fontes = []
    for i in r:
        fontes.append(i.text)
    return fontes

def get_book_autores(tree):
    n=tree.xpath('/article/back/ref-list/ref/nlm-citation[@citation-type="book"]/person-group[@person-group-type="author"]/name')
    nomes = []
    for i in n:
        nome = [i[0].text]+[i[1].text]
        nomes.append(nome)
    return nomes

def get_referencias(tree):     
    print('========== ref ==================')
    r=tree.xpath('/article/back/ref-list/ref/nlm-citation')
    fontes = []
    print(len(r))
    for i in r:
        citation_type = i.attrib['citation-type']
        if citation_type == 'book':
            autor = i[0]
        
        fonte=i.attrib['citation-type'],i[1].text
        fontes.append(fonte)
    return fontes

def get_texto(tree):     
    r=tree.xpath('/article/body')
    texto=''
    for b in r:
        if b.text != None:
            texto=texto+b.text
    return texto

print(get_abstract(tree))


Because the social science narrative has been extremely important in efforts to account for the process of Brazilian modernization, the study of social scientists themselves has become strategic to understanding Brazilian societys representation of self. This article examines graduate work in the social sciences, graduate students themselves, and their theses and dissertations. Research has been based on a survey conducted among candidates for masters degrees and doctorates, as well as on an analysis of doctoral dissertations defended between 1990 and 1997. The study reveals a discipline that is undergoing transformation a product of the process of democratization which its members has undergone and of the internalization of new parameters guiding cognitive production. While this may have implied a loss in the fields capacity to generalize, it has gained in capillarity, in plurality, and in capacity to understand. This study also pinpoints the limits of the discipline, determined by

In [5]:
#url_xml = 'file:///home/neilor/scielo_db/dados/artigos_xml/S1415-47572009000100001.xml'
url_xml = 'file:///home/neilor/scielo_db/dados/artigos_xml/S0104-62762016000100028.xml'

os.chdir(xml_dir)

import string as str

def sql_palavras_chave(palavras):
    n = len(palavras)
    pch = ["''","''","''","''","''"]
    for i in range(0,5):
        if i<n:
            palavra = palavras[i]
            palavra = palavra.replace("'", "''")
            pch[i]=("'"+palavra+"'")
    return (",".join(pch))

def sql_key_words(palavras):
    n = len(palavras)
    pch = ["''","''","''","''","''"]
    for i in range(0,5):
        if i<n:
            palavra = palavras[i]
            palavra = palavra.replace("'", "''")
            pch[i]=("'"+palavra+"'")
    return (",".join(pch))


def query_insert_dados_artigo_xml(scielo_id,arq_xml):
    try:
        tree = etree.parse(arq_xml)
        titulo = get_title(tree)
        titulo = titulo.replace("'", "''")
        
        resumo = get_resumo(tree)
        resumo = resumo.replace("'", "''")
        
        abstract = get_abstract(tree)
        abstract = abstract.replace("'", "''")
        #print(abstract)
        palavras_chave = get_palavras_chave(tree)
        #print(palavras_chave)
        palavras = sql_palavras_chave(palavras_chave)
        #print(palavras)
        
        key_words = get_key_words(tree)
        #print(key_words)
        words = sql_key_words(key_words)
        #print(words)
        
        values = "'"+scielo_id+"'"+",'"+titulo+"'"+",'"+resumo+"'"+","+palavras+",'"+abstract+"'"+","+words
        #print(values)
        sql = "insert into alacip.artigos_dados_xml  \
        (scielo_id,titulo,resumo,\
        palavra_chave1,palavra_chave2,palavra_chave3,palavra_chave4,palavra_chave5, \
        abstract,key_word1,key_word2,key_word3,key_word4,key_word5) values ("+values+");"

    except (etree.XMLSyntaxError, etree.ParseError):
        sql = "insert into alacip.artigos_dados_xml values ("+"'"+scielo_id+"','XML Invalido','','','');"
    return sql

#sql_palavras_chave(['a','b','c','d','e','f'])

#sql_palavras_chave(palavras_chave)
#print(query_insert_dados_artigo_xml('S0011-52581998000300001.xml','file:///home/neilor/scielo_db/dados/artigos_xml/S0011-52581998000300001.xml'))


insert into alacip.artigos_dados_xml          (scielo_id,titulo,resumo,        palavra_chave1,palavra_chave2,palavra_chave3,palavra_chave4,palavra_chave5,         abstract,key_word1,key_word2,key_word3,key_word4,key_word5) values ('S0011-52581998000300001.xml','Doutores e Teses em Ciências Sociais','','','','','','','Because the social science narrative has been extremely important in efforts to account for the process of Brazilian modernization, the study of social scientists themselves has become strategic to understanding Brazilian societys representation of self. This article examines graduate work in the social sciences, graduate students themselves, and their theses and dissertations. Research has been based on a survey conducted among candidates for masters degrees and doctorates, as well as on an analysis of doctoral dissertations defended between 1990 and 1997. The study reveals a discipline that is undergoing transformation a product of the process of democratization which 

In [5]:

sql_publishing_id_scielo = "select scielo_id from alacip.artigos_selecionados"

conn = pg.connect("dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'")
df = pd.read_sql_query(sql_publishing_id_scielo, conn)

for scielo_id in df['scielo_id']:
    xml_file = xml_dir+'/'+scielo_id +'.xml'
    file_path = Path(xml_file)
    #print(scielo_id)
    query = query_insert_dados_artigo_xml(scielo_id,xml_file)
    #print(values)
    #rint(query)
    with conn:
        with conn.cursor() as curs:
            curs.execute(query)
    
conn.close()
print("OK")




OK


In [6]:
os.chdir(xml_dir)
  

def query_update_xml_texto(scielo_id,arq_xml):
    try:
        #print(scielo_id)
        tree = etree.parse(arq_xml)
        texto = get_texto(tree)
        texto = texto.replace("'", "''")
        sql =  "update alacip.artigos_dados_xml set texto = '"+texto+"' where scielo_id like '"+scielo_id+"';"
    except (etree.XMLSyntaxError, etree.ParseError):
        sql =  "update alacip.artigos_dados_xml set texto = 'XML Invalido'"+" where scielo_id like '"+scielo_id+"';"
    return sql 

sql_publishing_id_scielo = "select scielo_id from alacip.artigos_selecionados"

conn = pg.connect("dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'")
df = pd.read_sql_query(sql_publishing_id_scielo, conn)

for scielo_id in df['scielo_id']:
    xml_file = xml_dir+'/'+scielo_id +'.xml'
    file_path = Path(xml_file)
    query = query_update_xml_texto(scielo_id,xml_file)
    with conn:
        with conn.cursor() as curs:
            curs.execute(query)
    
conn.close()
print("OK")

OK


In [7]:
query_update_tem_elemento_empirico = """
update alacip.artigos_dados_xml
set tem_tabela = '0',
    tem_quadro = '0',
    tem_grafico = '0',
    tem_diagrama = '0',
    tem_figura = '0',
    empirico = '0'
;

update alacip.artigos_dados_xml
set tem_tabela = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'tabela')
;

update alacip.artigos_dados_xml
set tem_tabela = '1'
WHERE to_tsvector('english', texto) @@ to_tsquery('english', 'table')
;

update alacip.artigos_dados_xml
set tem_grafico = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'grafico' || 'grafo')
;

update alacip.artigos_dados_xml
set tem_grafico = '1'
WHERE to_tsvector('english', texto) @@ to_tsquery('english', 'graphic' || 'plot' || 'graph')
;

update alacip.artigos_dados_xml
set tem_quadro = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'quadro')
;

update alacip.artigos_dados_xml
set tem_diagrama = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'diagrama')
;

update alacip.artigos_dados_xml
set tem_diagrama = '1'
WHERE to_tsvector('english', texto) @@ to_tsquery('english', 'diagram' || 'chart' )
;

update alacip.artigos_dados_xml
set tem_figura = '1'
WHERE to_tsvector('portuguese', texto) @@ to_tsquery('portuguese', 'figura')
;

update alacip.artigos_dados_xml
set empirico = '1' 
where tem_tabela = '1' or tem_quadro = '1' or tem_diagrama = '1' or tem_figura = '1'
;

"""

conn = pg.connect("dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'")
with conn:
    with conn.cursor() as curs:
        curs.execute(query_update_tem_elemento_empirico)
        conn.commit()
print("OK")


OK


In [8]:
conn = pg.connect("dbname='scielo_db' user='neilor' host='localhost' password='n1f2c3n1'")
df = pd.read_sql('select * from alacip.artigos_dados_xml;',conn)
df

,scielo_id,titulo,resumo,palavra_chave1,palavra_chave2,palavra_chave3,palavra_chave4,palavra_chave5,empirico,tem_tabela,...,tem_grafico,tem_diagrama,tem_figura,abstract,key_word1,key_word2,key_word3,key_word4,key_word5,texto
0,S0034-73291999000200016,Solução e Prevenção de Litígios Internacionais,,,,,,,0,0,...,0,0,0,,,,,,,"<p align=""right""><font face=""Verdana"" size=""2..."
1,S0102-64452000000300006,A comunidade cívica em Walter e Putnam,As concepções de comunidade cívica de Michael ...,,,,,,0,0,...,0,0,0,The article contrasts two conceptions of civic...,,,,,,"<p align=""right""><font size=""2"" face=""Verdana..."
2,S0011-52582008000200010,XML Invalido,,,,None,None,None,0,0,...,0,0,0,None,None,None,None,None,None,XML Invalido
3,S0011-52581997000200005,"Resultados Experimentais, Problemas Conceituai...",,,,,,,0,0,...,0,0,0,If we affirm that scientific knowledge advance...,scientific progress,research traditions,artificial intelligence,,,"<p><b>Resultados Experimentais, Problemas Con..."
4,S0011-52582013000100001,Nota editoial,,,,,,,0,0,...,0,0,0,,,,,,,"<p><font size=""4"" face=""Verdana""><B>Nota edit..."
5,S0011-52582014000200001,"Robert A. Dahl, econômico",,,,,,,1,0,...,0,0,0,,,,,,,"<p><font size=""4"" face=""Verdana""><B>Robert A...."
6,S0034-73291998000300004,Revista Brasileira de Política Internacional: ...,,,,,,,1,1,...,0,0,1,,,,,,,"<p align=""right""><font face=""Verdana"" size=""2..."
7,S0102-64452009000100001,Apresentação,,,,,,,0,0,...,0,0,0,,,,,,,"<p><font size=""4"" face=""Verdana, Arial, Helve..."
8,S0102-64452009000100008,Errata,,,,,,,0,0,...,0,0,0,,,,,,,"<p><font size=""4"" face=""Verdana, Arial, Helve..."
9,S0102-64452009000300008,Que fez São Tomás de Aquino diante de Karl Marx,O artigo aborda o diálogo de setores da Igreja...,Companhia de Jesus,Marxismo,Ceas,Igreja Católica,,0,0,...,0,0,0,,,,,,,"<p align=""right""><font size=""2"" face=""Verdana..."
